In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from causalml.inference.tf import DragonNet

In [ ]:
# Load dataset
df = pd.read_csv("C:/Users/User/Documents/GitHub/Health-impacts-of-air-pollution/MortData/GertPollMort30.csv", sep=';', header=0, index_col=0, parse_dates=True)


In [ ]:
# Drop rows with missing mortality data
df_clean = df.dropna(subset=['death_count'])

In [ ]:
# Define treatment: Days when PM2.5 > 40 & PM10 > 75
#df_clean['treatment'] = np.where((df_clean['pm2.5'] > 40) & (df_clean['pm10'] > 75), 1, 0)
df_clean['treatment'] = np.where((df_clean['pm2.5'] > 40), 1, 0)

In [ ]:
# Define features for propensity score matching
covariates = ['pm2.5', 'pm10', 'so2', 'no2', 'no', 'nox', 'o3', 'co', 'ws', 'wd', 'temp', 'relHum']

In [ ]:
# Fit Propensity Score Model (Logistic Regression)
ps_model = LogisticRegression(solver='liblinear')
df_clean = df_clean.dropna(subset=covariates)  # Drop missing values in covariates
ps_model.fit(df_clean[covariates], df_clean['treatment'])

In [ ]:
# Generate Propensity Scores
df_clean['propensity_score'] = ps_model.predict_proba(df_clean[covariates])[:, 1]

In [ ]:
# 1:1 Nearest Neighbor Matching
nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
control_group = df_clean[df_clean['treatment'] == 0][['propensity_score']]
treated_group = df_clean[df_clean['treatment'] == 1][['propensity_score']]
nn.fit(control_group)
distances, indices = nn.kneighbors(treated_group)

In [ ]:
# Create matched dataset
matched_control_indices = control_group.iloc[indices.flatten()].index
matched_data = pd.concat([df_clean.loc[matched_control_indices], df_clean[df_clean['treatment'] == 1]])

In [ ]:
#  DragonNet (Deep Learning Causal Model)**
  
# Prepare data for DragonNet
X_dragon = matched_data[covariates].astype(np.float32)
treatment_dragon = matched_data['treatment'].astype(np.float32)
y_dragon = matched_data['death_count'].astype(np.float32)


In [ ]:
# Standardize features
scaler = StandardScaler()
X_dragon_scaled = scaler.fit_transform(X_dragon)


In [ ]:
# Train DragonNet
dragonnet = DragonNet()
dragonnet.fit(X_dragon_scaled, treatment_dragon, y_dragon)

In [ ]:
# Predict treatment effects
treatment_effects_dragon = dragonnet.predict(X_dragon_scaled)
#ate_dragon = treatment_effects_dragon[:, 2].mean()
#print(f"\n🔹 DragonNet Estimated ATE: {ate_dragon}")

In [ ]:
#Here, you are calling .mean() directly on the entire treatment_effects array, which includes both columns. This can be misleading.
# To get the actual ATE as 𝜇1−𝜇0  , you need to subtract the two columns:
mu0 = treatment_effects_dragon[:, 0]
mu1 = treatment_effects_dragon[:, 1]
ite = mu1 - mu0  # Individual Treatment Effect for each sample
ate = ite.mean()
print("Average Treatment Effect (ATE):", ate)
# Otherwise, treatment_effects.mean() lumps together  𝜇1−𝜇0, which is not the difference.

In [ ]:
# 🔹 **Step 5: Visualizations**
# Plot mortality trends for treated vs. control groups
df_clean = df_clean.reset_index().rename(columns={'index': 'date'})
mortality_trend = df_clean.groupby(['date', 'treatment'])['death_count'].mean().unstack()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(mortality_trend.index, mortality_trend[0], label="Control (Low PM2.5)", color='blue')
plt.plot(mortality_trend.index, mortality_trend[1], label="Treated (High PM2.5)", color='red', linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Average Cardiovascular Mortality")
plt.title("Cardiovascular Mortality Trends: Low vs. High PM2.5 Days")
plt.legend()
plt.show()

In [ ]:
# Plot histogram of treatment effects
plt.hist(ite, bins=30, alpha=0.7, color='blue')
plt.title('Casual Effects at Gert Sibande', fontname="Times New Roman", size=28,fontweight="bold")
plt.xlabel('Cardiopulmonary casual Effect', fontname="Times New Roman", size=20,fontweight="bold")
plt.ylabel('Frequency', fontname="Times New Roman", size=20,fontweight="bold")
legend_properties = {'weight':'bold'}
plt.legend(prop=legend_properties)
plt.show()